In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re
from selenium import webdriver
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import matplotlib.pyplot as plt
import seaborn as sns
from selenium import webdriver
import nltk
import re
from nltk.tokenize import word_tokenize
from nltk.tokenize import WordPunctTokenizer
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
# matplotlib 한글화
import matplotlib as mpl
import matplotlib.font_manager as fm

mpl.rcParams['axes.unicode_minus'] = False

path = 'C:/Windows/Fonts/malgun.ttf'
font_name = fm.FontProperties(fname=path, size=50).get_name()
plt.rc('font', family=font_name)

In [3]:
df = pd.read_excel('./감성사전_편집.xlsx')
df = df.rename(columns={'Unnamed: 0':'단어'})
df.set_index('단어', drop=True, inplace=True)
df

,슬픔,분노,기쁨,긴장감,무서운,증오,차분히,섹시,브릿락,설렘,몽환
단어,,,,,,,,,,,
가련하다,1,0,0,0,0,0,0,0,0,0,0
가슴 아프다,1,0,0,0,0,0,0,0,0,0,0
가슴앓이,1,0,0,0,0,0,0,0,0,0,0
가엾다,1,0,0,0,0,0,0,0,0,0,0
각박하다,1,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
광대,1,0,0,0,1,0,0,0,0,0,0
한계였다,1,0,0,0,0,0,0,0,0,0,0
멸망했다,1,0,0,0,0,0,0,0,0,0,0


In [4]:
from selenium import webdriver
url = input('url을 입력하세요\n')

if 'https://novel.naver.com/webnovel/detail?' in url :
    print("웹소설 분석을 시작합니다.")
        
elif 'https://novel.naver.com/webnovel/detail?' not in url: 
    print("네이버 웹툰 웹소설의 url을 넣어주세요.")
    
res = requests.get(url)
driver = webdriver.Chrome()
driver.get(url)
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
path = '#content > div.section_area_viewer > div.viewer_container.dotum > div'
novel = soup.select(path)
text = novel[0].text

url을 입력하세요
https://novel.naver.com/webnovel/detail?novelId=963513&volumeNo=110
웹소설 분석을 시작합니다.


In [5]:
from konlpy.tag import Okt, Komoran, Kkma, Hannanum, Twitter
okt = Okt()

def text_clean(doc):
    text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣ ]', '', doc)
    return text
clean_text = text_clean(text)
tokens = okt.morphs(clean_text)
tokens = [j for j in tokens if len(j) > 1]
tokens

['가루',
 '미자',
 '넓은',
 '세상',
 '으로',
 '자유롭게',
 '보내주었던',
 '희는',
 '임신',
 '했다',
 '그리고',
 '시린',
 '눈발',
 '푸른',
 '하늘',
 '벚꽃',
 '처럼',
 '놓던',
 '이른',
 '아침',
 '희는',
 '자신',
 '했던',
 '약속',
 '가족',
 '에게',
 '지켰다',
 '강준',
 '에겐',
 '리틀',
 '주세',
 '희를',
 '신가',
 '말랐던',
 '귀한',
 '회장',
 '에겐',
 '증손녀',
 '이자',
 '연숙',
 '에겐',
 '손녀',
 '영준',
 '에겐',
 '사랑스러운',
 '여동생',
 '작은',
 '미소',
 '천사',
 '서예',
 '희가',
 '세상',
 '태어난',
 '이다',
 '가족',
 '만이',
 '아니라',
 '한신',
 '일원',
 '까지',
 '희의',
 '매력',
 '빠졌다',
 '내리',
 '아들',
 '태어나던',
 '신가',
 '탄생',
 '그만큼',
 '고귀했다',
 '아빠',
 '새까만',
 '눈동자',
 '닮은',
 '맞추며',
 '방긋',
 '웃으면',
 '누구',
 '살살',
 '녹아내렸다',
 '희도',
 '자신',
 '키워줄',
 '하고',
 '싶은',
 '마음껏',
 '하라',
 '했지만',
 '희는',
 '이번',
 '거절',
 '했다',
 '손주',
 '숙의',
 '마음',
 '온통',
 '사랑',
 '이란',
 '안다',
 '하지만',
 '있는데도',
 '영준',
 '거의',
 '맡아서',
 '키운',
 '숙의',
 '체력',
 '안타깝게도',
 '아니었다',
 '어머니',
 '볼게요',
 '즈음',
 '복직',
 '하니까',
 '어머니',
 '봐주세요',
 '물론',
 '같이',
 '보신다는',
 '조건',
 '에요',
 '한텐',
 '아이',
 '소중하지만',
 '어머니',
 '소중한',
 '아시잖아요',
 '건강하게',
 '오래오래',
 '사셔야죠',
 '간절한',
 '부탁',
 '숙이',
 '드디어'

In [6]:
sent_words = []

for tok in tokens:
    try:
        ser = df.loc[tok]
        sent_words.append(ser[df.loc[tok] == 1].index[0])

    except:
        pass
pd.Series(sent_words).value_counts()

설렘     32
섹시     27
기쁨     23
몽환     16
슬픔      9
차분히     9
브릿락     5
분노      3
긴장감     3
무서운     2
증오      2
dtype: int64

In [10]:
counter = {}
for i in tokens:
    for j in df.index:
        if i == j:
            if i in counter:
                counter[i] += 1
            else:
                counter[i] = 1
counter

{'자유롭게': 1,
 '귀한': 1,
 '사랑스러운': 2,
 '미소': 4,
 '천사': 4,
 '매력': 2,
 '방긋': 1,
 '거절': 1,
 '사랑': 4,
 '조건': 1,
 '소중한': 1,
 '눈치': 1,
 '통증': 1,
 '아파': 1,
 '앓는': 1,
 '행복한': 1,
 '가슴': 3,
 '익숙한': 1,
 '따뜻한': 1,
 '탐스러운': 1,
 '심장': 2,
 '말로': 1,
 '표현': 1,
 '감정': 1,
 '보다': 3,
 '고통': 1,
 '이해': 1,
 '한결같이': 4,
 '여자': 3,
 '반짝': 1,
 '한숨': 1,
 '말똥말똥': 1,
 '뛰어나': 1,
 '의심': 1,
 '조심': 1,
 '부드럽게': 1,
 '귀엽게': 1,
 '특별히': 1,
 '따가운': 1,
 '시선': 4,
 '눈빛': 7,
 '욕망': 3,
 '참고': 1,
 '해소': 1,
 '영양가': 1,
 '빨리': 1,
 '느리게': 1,
 '고생': 1,
 '편히': 1,
 '두려워': 1,
 '숨소리': 2,
 '화음': 1,
 '나쁘지': 1,
 '흐르고': 1,
 '믿을': 1,
 '감탄': 2,
 '칭찬': 1,
 '좋고': 3,
 '부러운': 1,
 '애정': 1,
 '떨어지는': 1,
 '싫다는': 1,
 '죽음': 1,
 '뜨거운': 1,
 '한결같은': 1,
 '아름다운': 1,
 '우아하고': 1,
 '정교한': 1,
 '날카로운': 1,
 '웃음': 2,
 '특권': 2,
 '새끼': 1,
 '떨어지고': 1,
 '인정': 1,
 '딸바보': 1,
 '신경': 1,
 '한다는': 1,
 '진짜': 1,
 '멋진': 2,
 '거짓말': 1}

In [11]:
sent_words = []

for tok in tokens:
    try:
        print(df.loc[tok])
        sent_word = df[df.loc[tok] == 1].values
        print('token :', tok, 'sent_word :', sent_word)

    except:
        pass

슬픔     0
분노     0
기쁨     1
긴장감    0
무서운    0
증오     0
차분히    0
섹시     0
브릿락    0
설렘     0
몽환     0
Name: 자유롭게, dtype: int64
슬픔     0
분노     0
기쁨     1
긴장감    0
무서운    0
증오     0
차분히    0
섹시     0
브릿락    0
설렘     0
몽환     0
Name: 귀한, dtype: int64
슬픔     0
분노     0
기쁨     0
긴장감    0
무서운    0
증오     0
차분히    0
섹시     0
브릿락    0
설렘     1
몽환     0
Name: 사랑스러운, dtype: int64
슬픔     0
분노     0
기쁨     0
긴장감    0
무서운    0
증오     0
차분히    0
섹시     0
브릿락    0
설렘     1
몽환     0
Name: 미소, dtype: int64
슬픔     0
분노     0
기쁨     0
긴장감    0
무서운    0
증오     0
차분히    0
섹시     0
브릿락    0
설렘     0
몽환     1
Name: 천사, dtype: int64
슬픔     0
분노     0
기쁨     0
긴장감    0
무서운    0
증오     0
차분히    0
섹시     1
브릿락    0
설렘     0
몽환     0
Name: 매력, dtype: int64
슬픔     0
분노     0
기쁨     1
긴장감    0
무서운    0
증오     0
차분히    0
섹시     0
브릿락    0
설렘     0
몽환     0
Name: 방긋, dtype: int64
슬픔     1
분노     0
기쁨     0
긴장감    0
무서운    0
증오     1
차분히    0
섹시     0
브릿락    0
설렘     0
몽환     0
Name: 거절, dtype: int64
슬픔     0
분노     0
기

C:\Users\bitcamp\AppData\Local\Temp/ipykernel_4072/2794359773.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  sent_word = df[df.loc[tok] == 1].values


### song_dataframe

In [12]:
sad = pd.DataFrame(columns = ['노래','슬픔', '분노', '기쁨', '긴장감', '무서운', '증오', \
                                  '차분히', '섹시', '브릿락', '설렘', '몽환'])
driver = webdriver.Chrome() 
url = 'https://www.melon.com/mymusic/dj/mymusicdjplaylistview_inform.htm?plylstSeq=484083307'
driver.get(url)
path = '#frm > div'
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

for i in range(0,50) :
    num = soup.select('')
    songs = soup.select('div.ellipsis.rank01 > span > a')[i].text
    singers = soup.select('div.ellipsis.rank02 > span > a')[i].text
    sad = sad.append({'노래' :(songs,singers)},ignore_index=True)
sad['슬픔'] = sad['슬픔'].fillna(10)

WebDriverException: Message: unknown error: cannot determine loading status
from disconnected: received Inspector.detached event
  (Session info: chrome=100.0.4896.127)
Stacktrace:
Backtrace:
	Ordinal0 [0x00C87413+2389011]
	Ordinal0 [0x00C19F61+1941345]
	Ordinal0 [0x00B0C658+837208]
	Ordinal0 [0x00AFE87E+780414]
	Ordinal0 [0x00AFE46F+779375]
	Ordinal0 [0x00AFDA86+776838]
	Ordinal0 [0x00AFC995+772501]
	Ordinal0 [0x00AFCF98+774040]
	Ordinal0 [0x00B08C20+822304]
	Ordinal0 [0x00AFE27D+778877]
	Ordinal0 [0x00AFEE3D+781885]
	Ordinal0 [0x00AFE48A+779402]
	Ordinal0 [0x00AFDA86+776838]
	Ordinal0 [0x00AFC995+772501]
	Ordinal0 [0x00AFCE6D+773741]
	Ordinal0 [0x00B0DE5A+843354]
	Ordinal0 [0x00B649BD+1198525]
	Ordinal0 [0x00B5414C+1130828]
	Ordinal0 [0x00B64302+1196802]
	Ordinal0 [0x00B53F66+1130342]
	Ordinal0 [0x00B2E546+976198]
	Ordinal0 [0x00B2F456+980054]
	GetHandleVerifier [0x00E39632+1727522]
	GetHandleVerifier [0x00EEBA4D+2457661]
	GetHandleVerifier [0x00D1EB81+569713]
	GetHandleVerifier [0x0

In [ ]:
happy = pd.DataFrame(columns = ['노래','슬픔', '분노', '기쁨', '긴장감', '무서운', '증오', \
                                  '차분히', '섹시', '브릿락', '설렘', '몽환'])
driver = webdriver.Chrome() 
url = 'https://www.melon.com/mymusic/dj/mymusicdjplaylistview_inform.htm?plylstSeq=506175413'
driver.get(url)
path = '#frm > div'
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

for i in range(0,50) :

    songs = soup.select('div.ellipsis.rank01 > span > a')[i].text
    singers = soup.select('div.ellipsis.rank02 > span > a')[i].text
    happy = happy.append({'노래' :(songs,singers)},ignore_index=True)
happy['기쁨'] = happy['기쁨'].fillna(10)

In [34]:
anger = pd.DataFrame(columns = ['노래','슬픔', '분노', '기쁨', '긴장감', '무서운', '증오', \
                                  '차분히', '섹시', '브릿락', '설렘', '몽환'])
driver = webdriver.Chrome() 
url = 'https://www.melon.com/mymusic/dj/mymusicdjplaylistview_inform.htm?plylstSeq=490101618'
driver.get(url)
path = '#frm > div'
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

for i in range(0,38) :

    songs = soup.select('div.ellipsis.rank01 > span > a')[i].text
    singers = soup.select('div.ellipsis.rank02 > span > a')[i].text
    anger = anger.append({'노래' :(songs,singers)},ignore_index=True)
anger['분노'] = anger['분노'].fillna(10)

In [35]:
nerve = pd.DataFrame(columns = ['노래','슬픔', '분노', '기쁨', '긴장감', '무서운', '증오', \
                                  '차분히', '섹시', '브릿락', '설렘', '몽환'])
driver = webdriver.Chrome() 
url = 'https://www.melon.com/mymusic/dj/mymusicdjplaylistview_inform.htm?plylstSeq=412763124'
driver.get(url)
path = '#frm > div'
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

for i in range(0,50) :

    songs = soup.select('div.ellipsis.rank01 > span > a')[i].text
    singers = soup.select('div.ellipsis.rank02 > span > a')[i].text
    nerve = nerve.append({'노래' :(songs,singers)},ignore_index=True)
nerve['긴장감'] = nerve['긴장감'].fillna(10)

In [36]:
scary = pd.DataFrame(columns = ['노래','슬픔', '분노', '기쁨', '긴장감', '무서운', '증오', \
                                  '차분히', '섹시', '브릿락', '설렘', '몽환'])
driver = webdriver.Chrome() 
url = 'https://www.melon.com/mymusic/dj/mymusicdjplaylistview_inform.htm?plylstSeq=467106196'
driver.get(url)
path = '#frm > div'
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

for i in range(0,31) :

    songs = soup.select('div.ellipsis.rank01 > span > a')[i].text
    singers = soup.select('div.ellipsis.rank02 > span > a')[i].text
    scary = scary.append({'노래' :(songs,singers)},ignore_index=True)
scary['무서운'] = scary['무서운'].fillna(10)

In [37]:
hatred = pd.DataFrame(columns = ['노래','슬픔', '분노', '기쁨', '긴장감', '무서운', '증오', \
                                  '차분히', '섹시', '브릿락', '설렘', '몽환'])
driver = webdriver.Chrome() 
url = 'https://www.melon.com/mymusic/dj/mymusicdjplaylistview_inform.htm?plylstSeq=426635404'
driver.get(url)
path = '#frm > div'
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

for i in range(0,18) :

    songs = soup.select('div.ellipsis.rank01 > span > a')[i].text
    singers = soup.select('div.ellipsis.rank02 > span > a')[i].text
    hatred = hatred.append({'노래' :(songs,singers)},ignore_index=True)
hatred['증오'] = hatred['증오'].fillna(10)

In [38]:
calm = pd.DataFrame(columns = ['노래','슬픔', '분노', '기쁨', '긴장감', '무서운', '증오', \
                                  '차분히', '섹시', '브릿락', '설렘', '몽환'])
driver = webdriver.Chrome() 
url = 'https://www.melon.com/mymusic/dj/mymusicdjplaylistview_inform.htm?plylstSeq=506846192'
driver.get(url)
path = '#frm > div'
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

for i in range(0,47) :

    songs = soup.select('div.ellipsis.rank01 > span > a')[i].text
    singers = soup.select('div.ellipsis.rank02 > span > a')[i].text
    calm = calm.append({'노래' :(songs,singers)},ignore_index=True)
calm['차분히'] = calm['차분히'].fillna(10)

In [39]:
sexy = pd.DataFrame(columns = ['노래','슬픔', '분노', '기쁨', '긴장감', '무서운', '증오', \
                                  '차분히', '섹시', '브릿락', '설렘', '몽환'])
driver = webdriver.Chrome() 
url = 'https://www.melon.com/mymusic/dj/mymusicdjplaylistview_inform.htm?plylstSeq=443836057'
driver.get(url)
path = '#frm > div'
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

for i in range(0,50) :

    songs = soup.select('div.ellipsis.rank01 > span > a')[i].text
    singers = soup.select('div.ellipsis.rank02 > span > a')[i].text
    sexy = sexy.append({'노래' :(songs,singers)},ignore_index=True)
sexy['섹시'] = sexy['섹시'].fillna(10)

In [40]:
britrock = pd.DataFrame(columns = ['노래','슬픔', '분노', '기쁨', '긴장감', '무서운', '증오', \
                                  '차분히', '섹시', '브릿락', '설렘', '몽환'])
driver = webdriver.Chrome() 
url = 'https://www.melon.com/mymusic/dj/mymusicdjplaylistview_inform.htm?plylstSeq=508377011'
driver.get(url)
path = '#frm > div'
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

for i in range(0,44) :

    songs = soup.select('div.ellipsis.rank01 > span > a')[i].text
    singers = soup.select('div.ellipsis.rank02 > span > a')[i].text
    britrock = britrock.append({'노래' :(songs,singers)},ignore_index=True)
britrock['브릿락'] = britrock['브릿락'].fillna(10)

In [41]:
dokidoki = pd.DataFrame(columns = ['노래','슬픔', '분노', '기쁨', '긴장감', '무서운', '증오', \
                                  '차분히', '섹시', '브릿락', '설렘', '몽환'])
driver = webdriver.Chrome() 
url = 'https://www.melon.com/mymusic/dj/mymusicdjplaylistview_inform.htm?plylstSeq=489567062'
driver.get(url)
path = '#frm > div'
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

for i in range(0,50) :

    songs = soup.select('div.ellipsis.rank01 > span > a')[i].text
    singers = soup.select('div.ellipsis.rank02 > span > a')[i].text
    dokidoki = dokidoki.append({'노래' :(songs,singers)},ignore_index=True)
dokidoki['설렘'] = dokidoki['설렘'].fillna(10)

In [42]:
song = pd.concat([happy, sad, anger, dokidoki, britrock, sexy, calm, hatred, scary, nerve])
song.reset_index(inplace=True,drop = True)
song

,노래,슬픔,분노,기쁨,긴장감,무서운,증오,차분히,섹시,브릿락,설렘,몽환
0,"(바야흐로 사랑의 계절, 이한철)",NaN,NaN,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"(사랑을 시작해도 되겠습니까, 박새별)",NaN,NaN,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"(피너츠 송, 모던 쥬스)",NaN,NaN,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"(일단은 달달한 노래 (Feat. Gamjay, 임미소), 상상밴드)",NaN,NaN,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"(Think About' Chu, Rooftop Gigs)",NaN,NaN,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
423,"(eclipse (이클립스), Steelheart)",NaN,NaN,NaN,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN
424,"(This Is The Beginning, Patrick Joseph)",NaN,NaN,NaN,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN
425,"(Superbad (11:34), Marco Beltrami)",NaN,NaN,NaN,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN
426,"(비극 3 (진실), Travie McCoy)",NaN,NaN,NaN,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### lyrics_dataframe

In [46]:
text = '''
우연히 내게 오나봐
봄 향기가 보여
너도 같이 오나봐
저 멀리서 니 향기가
설레는 코끝에 나의 입술에
괜찮은 느낌 이 떨림
나도 몰래 우연히 봄
어 어느새
겨울 지나 봄이야
여전히 난 너
앞에선 돌이야
난 아직 이게
믿기지가 않지만
내 왼손은
지금까지도 너의 향기가
미묘하게 흘렀던 분위기에
아직까지 난 가까스로 숨 쉬네
무대 위완 다르게 니 눈을 피해
고개를 돌렸던
내 모습에 한숨 쉬네
오랜만에 느껴지는
이 떨림이 날 단순하게 만들어
딱 너만 아는 거리지
다 고쳤다고 생각했던 버벅임이
또 도져서 준비했던
말을 잊어버리지
난 주워 담지 못할
말은 절대 안 해
원하는 걸 말해봐
널 위해서만 할게
너 빼곤 다
색칠할 수 있어 까맣게
천천히 갈게 조금 더 가깝게
우연히 내게 오나봐
봄 향기가 보여
너도 같이 오나봐
저 멀리서 니 향기가
설레는 코끝에 나의 입술에
괜찮은 느낌 이 떨림
나도 몰래 우연히 봄
우연히 봄
어 아직까지 향기가 짙네
발걸음이 가벼워
집에 가는 길엔
더 가까워질 너와 나를
상상하는 내 모습이
오글거려 몸서리치네
어 어 머릿속이
하얘지기 때문에
하루 종일 날씨
얘기만 반복하게 돼
오로지 난 너 하나 때문에
다른 것들에겐 무감각하게 돼
정적이 만드는 긴장감은
오히려 설레어
나를 미소 짓게 만들어
모른 척 하려 했던
니 옆의 남자들은
흐릿했던 내 눈 앞에
불을 키게 만들어
겨울은 유난히도 추웠고
다시 돌아온
그토록 기다렸던 봄
움직이지 못하도록
잡고 있고 싶어
날 더 느낄 수 있게
안고 있고 싶어
우연히 내게 오나봐
봄 향기가 보여
너도 같이 오나봐
저 멀리서 니 향기가
설레는 코끝에 나의 입술에
괜찮은 느낌 이 떨림
나도 몰래 우연히 봄
I love u so I love u
너무 쉬운 걸
그래도 나 참고 있을게
난 여자이니까
하루 더 기다려
바보야 내게 말해봐
니 마음도 보여
갖고 싶다고 해봐
더 이상은 감추지 마
어느새 내 앞에 이젠 내 앞에
괜찮은 느낌 이 떨림
나도 몰래 우연히 봄'''

In [48]:
from konlpy.tag import Okt, Komoran, Kkma, Hannanum, Twitter
okt = Okt()
lyrics = okt.morphs(text)
lyrics = [j for j in lyrics if len(j) > 1]
lyrics

['우연히',
 '내게',
 '향기',
 '보여',
 '같이',
 '멀리',
 '향기',
 '설레는',
 '코끝',
 '입술',
 '괜찮은',
 '느낌',
 '떨림',
 '나도',
 '몰래',
 '우연히',
 '어느새',
 '겨울',
 '지나',
 '이야',
 '여전히',
 '에선',
 '돌이',
 '아직',
 '믿기지가',
 '않지만',
 '왼손',
 '지금',
 '까지도',
 '향기',
 '미묘하게',
 '흘렀던',
 '분위기',
 '아직',
 '까지',
 '가까스로',
 '무대',
 '위완',
 '다르게',
 '피해',
 '고개',
 '돌렸던',
 '모습',
 '한숨',
 '오랜',
 '느껴지는',
 '떨림이',
 '단순하게',
 '만들어',
 '아는',
 '거리',
 '고쳤다고',
 '생각',
 '했던',
 '버벅임',
 '져서',
 '준비',
 '했던',
 '잊어버리지',
 '주워',
 '담지',
 '절대',
 '원하는',
 '해봐',
 '해서만',
 '할게',
 '빼곤',
 '색칠',
 '있어',
 '까맣게',
 '천천히',
 '갈게',
 '조금',
 '가깝게',
 '우연히',
 '내게',
 '향기',
 '보여',
 '같이',
 '멀리',
 '향기',
 '설레는',
 '코끝',
 '입술',
 '괜찮은',
 '느낌',
 '떨림',
 '나도',
 '몰래',
 '우연히',
 '우연히',
 '아직',
 '까지',
 '향기',
 '짙네',
 '발걸음',
 '가벼워',
 '가는',
 '가까워질',
 '나를',
 '상상',
 '하는',
 '모습',
 '오글거려',
 '몸서리',
 '치네',
 '머릿속',
 '하얘지기',
 '때문',
 '하루',
 '종일',
 '날씨',
 '얘기',
 '반복',
 '하게',
 '오로지',
 '하나',
 '때문',
 '다른',
 '에겐',
 '무감각',
 '하게',
 '정적',
 '만드는',
 '긴장감',
 '오히려',
 '설레어',
 '나를',
 '미소',
 '짓게',
 '만들어',
 '모른',
 '하려',
 '했던',
 '남자'

In [49]:
counter={}
for i in lyrics:
    for j in df.index:
        if i == j:
            if i in counter:
                counter[i] += 1
            else:
                counter[i] = 1
counter

{'향기': 8,
 '설레는': 3,
 '느낌': 4,
 '떨림': 4,
 '몰래': 4,
 '분위기': 1,
 '피해': 1,
 '한숨': 1,
 '가깝게': 1,
 '설레어': 1,
 '미소': 1,
 '참고': 1,
 '여자': 1}

In [51]:
sent_words = []

for tok in lyrics:
    try:
        ser = df.loc[tok]
        sent_words.append(ser[df.loc[tok] == 1].index[0])

    except:
        pass
pd.Series(sent_words).value_counts()

설렘     14
긴장감     8
섹시      5
슬픔      2
몽환      1
차분히     1
dtype: int64

In [ ]:
sad = pd.DataFrame(columns = ['노래','슬픔', '분노', '기쁨', '긴장감', '무서운', '증오','차분히', '섹시', '브릿락', '설렘', '몽환'])
temps = []
driver = webdriver.Chrome() 
import time
for i in range(1,952,50) :
    url = 'https://www.melon.com/mymusic/dj/mymusicdjplaylistview_inform.htm?plylstSeq=403014132#params%5BplylstSeq%5D=403014132&po=pageObj&startIndex={}'.format(i)
    driver.get(url)
    time.sleep(0.15)
    path = '#frm > div'
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    for i in range(0,50) :
        songs = soup.select('div.ellipsis.rank01 > span > a')[i].text
        singers = soup.select('div.ellipsis.rank02 > span > a')[i].text
        temps.append((songs, singers))
        
sad['노래'] = temps